## 1 Preparación de datos

In [44]:
#importar librerias
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.metrics import mean_squared_error

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from catboost import CatBoostRegressor

In [2]:
#importar el dataset
df = pd.read_csv('/Users/whitneyrios/PYTHON/My projects py/proyecto 12/Rusty-Bargain/dataset/car_data.csv')

In [3]:
#visualizar los datos 
df.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,24/03/2016 11:52,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,24/03/2016 00:00,0,70435,07/04/2016 03:16
1,24/03/2016 10:58,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,24/03/2016 00:00,0,66954,07/04/2016 01:46
2,14/03/2016 12:52,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,14/03/2016 00:00,0,90480,05/04/2016 12:47
3,17/03/2016 16:54,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,17/03/2016 00:00,0,91074,17/03/2016 17:40
4,31/03/2016 17:25,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,31/03/2016 00:00,0,60437,06/04/2016 10:17


In [4]:
#visualizando la informacion general del df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [5]:
#viendo la descripcion de los datos 
df.describe()

,Price,RegistrationYear,Power,Mileage,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


In [6]:
#comprobando datos ausentes
df.isna().sum()

DateCrawled              0
Price                    0
VehicleType          37490
RegistrationYear         0
Gearbox              19833
Power                    0
Model                19705
Mileage                  0
RegistrationMonth        0
FuelType             32895
Brand                    0
NotRepaired          71154
DateCreated              0
NumberOfPictures         0
PostalCode               0
LastSeen                 0
dtype: int64

Se encontraron algunos datos ausentes en cinco columnas los cuales eliminaré a continuación.

In [7]:
#eliminando valores ausentes
df = df.dropna().reset_index(drop=True)

In [8]:
#verificando el df luego de la eliminación
df.isna().sum()

DateCrawled          0
Price                0
VehicleType          0
RegistrationYear     0
Gearbox              0
Power                0
Model                0
Mileage              0
RegistrationMonth    0
FuelType             0
Brand                0
NotRepaired          0
DateCreated          0
NumberOfPictures     0
PostalCode           0
LastSeen             0
dtype: int64

In [9]:
#verificando valores duplicados
df.duplicated().sum()

247

Encontré algunos valores duplicados, los cuales eliminaré del dataset.

In [10]:
#eliminar valores duplicados 
df = df.drop_duplicates().reset_index(drop=True)

In [11]:
#verificando eliminación de duplicados
df.duplicated().sum()

0

In [12]:
#verificar el dataset luego de eliminar valores ausentes y duplicados 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 245567 entries, 0 to 245566
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        245567 non-null  object
 1   Price              245567 non-null  int64 
 2   VehicleType        245567 non-null  object
 3   RegistrationYear   245567 non-null  int64 
 4   Gearbox            245567 non-null  object
 5   Power              245567 non-null  int64 
 6   Model              245567 non-null  object
 7   Mileage            245567 non-null  int64 
 8   RegistrationMonth  245567 non-null  int64 
 9   FuelType           245567 non-null  object
 10  Brand              245567 non-null  object
 11  NotRepaired        245567 non-null  object
 12  DateCreated        245567 non-null  object
 13  NumberOfPictures   245567 non-null  int64 
 14  PostalCode         245567 non-null  int64 
 15  LastSeen           245567 non-null  object
dtypes: int64(7), object(

## 2 Entrenamiento del modelo 

In [13]:
#funcion para escalar caracteristicas numericas en regresion lineal 
def scaler(data):
    scaler = StandardScaler()
    scaler.fit_transform(data)
    
    data_scaled = scaler.transform(data)
   
    return data_scaled

In [14]:
#funcion para separar los conjuntos de datos
def func_train_test_split(X, y, test_size):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=12345)
    
    return X_train, X_test, y_train, y_test

In [15]:
#función para encontrar el valor rmse
def rmse(y_pred, y):
    mse = mean_squared_error(y_pred, y)
    return mse ** 0.5

### 2.1.1 Modelo de regresión lineal

In [16]:
#Dataframe con columnas numéricas (eliminando columnas innecesarias para prediccion)
data_num_columns = df.drop(['DateCrawled', 'DateCreated', 'LastSeen', 'VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired'], axis=1)

In [17]:
#visualizar el nuevo dataframe numérico
data_num_columns.head()

,Price,RegistrationYear,Power,Mileage,RegistrationMonth,NumberOfPictures,PostalCode
0,1500,2001,75,150000,6,0,91074
1,3600,2008,69,90000,7,0,60437
2,650,1995,102,150000,10,0,33775
3,2200,2004,109,150000,8,0,67112
4,0,1980,50,40000,7,0,19348


In [18]:
#codificar columnas categóricas del df
data_ohe_columns = pd.get_dummies(df[['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']]).astype('int')
data_ohe_columns.head()

,VehicleType_bus,VehicleType_convertible,VehicleType_coupe,VehicleType_other,VehicleType_sedan,VehicleType_small,VehicleType_suv,VehicleType_wagon,Gearbox_auto,Gearbox_manual,...,Brand_skoda,Brand_smart,Brand_subaru,Brand_suzuki,Brand_toyota,Brand_trabant,Brand_volkswagen,Brand_volvo,NotRepaired_no,NotRepaired_yes
0,0,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,1,0,1,0
1,0,0,0,0,0,1,0,0,0,1,...,1,0,0,0,0,0,0,0,1,0
2,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
3,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,1,0,1,0


In [19]:
#escalar columnas numericas
data_num_scaled = scaler(data_num_columns)

#concatenar un dataframe con caracteristicas escaladas y codificadas 
data_ohe_scaled = pd.concat([pd.DataFrame(data_num_scaled, columns=data_num_columns.columns)] + [data_ohe_columns], axis=1)

data_ohe_scaled.head()

,Price,RegistrationYear,Power,Mileage,RegistrationMonth,NumberOfPictures,PostalCode,VehicleType_bus,VehicleType_convertible,VehicleType_coupe,...,Brand_skoda,Brand_smart,Brand_subaru,Brand_suzuki,Brand_toyota,Brand_trabant,Brand_volkswagen,Brand_volvo,NotRepaired_no,NotRepaired_yes
0,-0.768404,-0.311174,-0.322499,0.612277,-0.051555,0.0,1.532972,0,0,0,...,0,0,0,0,0,0,1,0,1,0
1,-0.323343,0.824328,-0.365526,-1.005870,0.235819,0.0,0.347318,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,-0.948547,-1.284461,-0.128878,0.612277,1.097944,0.0,-0.684503,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,-0.620050,0.175470,-0.078680,0.612277,0.523194,0.0,0.605641,0,1,0,...,0,0,0,0,0,0,0,0,1,0
4,-1.086304,-3.717679,-0.501778,-2.354325,0.235819,0.0,-1.242829,0,0,0,...,0,0,0,0,0,0,1,0,1,0


In [20]:
#separar las caracteristicas y el objetivo 
X_lr = data_ohe_scaled.drop(['Price'], axis=1)
y_lr = data_ohe_scaled['Price']

In [21]:
#segmentar el df en conjuntos de entrenamiento y prueba 
X_train_lr, X_test_lr, y_train_lr, y_test_lr = func_train_test_split(X_lr, y_lr, 0.30)

In [22]:
#entrenar un modelo de regresión lineal 
model_lr = LinearRegression()
model_lr.fit(X_train_lr, y_train_lr)

LinearRegression()

In [23]:
#encontrar el valor rmse en regresion lineal
predicted_test_lr = model_lr.predict(X_test_lr)

model_lr_rmse = rmse(predicted_test_lr, y_test_lr)

print('Valor rmse para regresión lineal:', model_lr_rmse)

Valor rmse para regresión lineal: 0.5741409420574772


La regresión lineal tarda alrededor de 7.7 segundos en el entrenemiento. En cuanto a la métrica de evaluación, tenemos un valor de 0.57, sin  hiperparámetros. 

### 2.1.2 Modelos basados en árboles

In [24]:
#seleccionar columnas categoricas del df
data_categorical_columns = df[['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']]

In [25]:
#codificar caracteristicas categoricas con  para arboles
encoder = OrdinalEncoder()

data_ordinal = pd.DataFrame(encoder.fit_transform(data_categorical_columns), columns=data_categorical_columns.columns)

data_ordinal.head()

,VehicleType,Gearbox,Model,FuelType,Brand,NotRepaired
0,5.0,1.0,116.0,6.0,37.0,0.0
1,5.0,1.0,101.0,2.0,31.0,0.0
2,4.0,1.0,11.0,6.0,2.0,1.0
3,1.0,1.0,8.0,6.0,25.0,0.0
4,4.0,1.0,166.0,6.0,37.0,0.0


In [26]:
#concatenar un dataframe para caracteristicas escaladas y data_ordinal
data_ordinal_scaled = pd.concat([pd.DataFrame(data_num_scaled, columns=data_num_columns.columns)] + [data_ordinal], axis=1)

data_ordinal_scaled.head()

,Price,RegistrationYear,Power,Mileage,RegistrationMonth,NumberOfPictures,PostalCode,VehicleType,Gearbox,Model,FuelType,Brand,NotRepaired
0,-0.768404,-0.311174,-0.322499,0.612277,-0.051555,0.0,1.532972,5.0,1.0,116.0,6.0,37.0,0.0
1,-0.323343,0.824328,-0.365526,-1.005870,0.235819,0.0,0.347318,5.0,1.0,101.0,2.0,31.0,0.0
2,-0.948547,-1.284461,-0.128878,0.612277,1.097944,0.0,-0.684503,4.0,1.0,11.0,6.0,2.0,1.0
3,-0.620050,0.175470,-0.078680,0.612277,0.523194,0.0,0.605641,1.0,1.0,8.0,6.0,25.0,0.0
4,-1.086304,-3.717679,-0.501778,-2.354325,0.235819,0.0,-1.242829,4.0,1.0,166.0,6.0,37.0,0.0


In [27]:
#separar caracteristicas y objetivo
X_ordinal_scaled = data_ordinal_scaled.drop(['Price'], axis=1)
y_ordinal_scaled = data_ordinal_scaled['Price']

In [28]:
#segmentar en conjuntos de entrenamiento y prueba
X_train_or_scaled, X_test_or_scaled, y_train_or_scaled, y_test_or_scaled = func_train_test_split(X_ordinal_scaled, y_ordinal_scaled, 0.30)

### 2.1.2.1 Modelo de bosque aleatorio (regresión)

(Buscando los hiperparámetros con GridSearch)

In [29]:
#bosque aleatorio regresion
#model_rfr = RandomForestRegressor(random_state=12345)

#params = {
    #'n_estimators' : [5, 8, 10],
    #'max_depth' : [5, 6, 8],
    ##'min_samples_split' : [2, 3],#
#} 
 
#buscar hiperparametros
#grid = GridSearchCV(estimator=model_rfr, param_grid=params, cv=5)

#grid.fit(X_train_or_scaled, y_train_or_scaled)

In [30]:
#mostrar los mejores hiperparámetros del modelo
#grid.best_params_

(Construyendo el modelo de bosque aleatorio con los parámetros del grid)

In [31]:
#construir un bosque aletorio con los hiperparámetros de grid
model_rfr = RandomForestRegressor(random_state=12345, 
                                  max_depth=8,
                                  n_estimators=10,
                                  min_samples_split=3 
                                  )

#entrenar el modelo 
model_rfr.fit(X_train_or_scaled, y_train_or_scaled)

RandomForestRegressor(max_depth=8, min_samples_split=3, n_estimators=10,
                      random_state=12345)

In [32]:
#calcular el valor rmse
predicted_test_rfr = model_rfr.predict(X_test_or_scaled)

model_rfr_rmse = rmse(predicted_test_rfr, y_test_or_scaled)

print('Valor de rmse en bosque aleatorio:', model_rfr_rmse)

Valor de rmse en bosque aleatorio: 0.4445110229948601


El bosque aleatorio empleo menos tiempo en el entenamiento que la regresión lineal, y el valor rmse mejora bastante, 0.44.

### 2.1.2.2 Modelo de árbol de decisión (regresión)

(Buscando los hiperparámetros con GridSearch)

In [33]:
#arbol de decision de regresion

#model_dtr = DecisionTreeRegressor(random_state=54321)


#params = {
    #'max_depth' : [5, 6, 8],
    #'splitter' : ['best', 'random'],
    #'min_samples_split' : [2, 3],
    #'min_samples_leaf' : [2, 3],
    #'max_features' : [3, 4],
#} 

#buscar hiperparametros 
#grid = GridSearchCV(estimator=model_dtr, param_grid=params, cv=5)

#grid.fit(X_train_or_scaled, y_train_or_scaled)

In [34]:
#mostrar los mejores hiperparametros 
#grid.best_estimator_

(Construyendo el modelo de bosque aleatorio con los parámetros del grid)

In [35]:
#construir un modelo de arbol de regresion
model_dtr = DecisionTreeRegressor(max_depth=8, max_features=4, min_samples_leaf=2,
                      random_state=54321)

model_dtr.fit(X_train_or_scaled, y_train_or_scaled)

DecisionTreeRegressor(max_depth=8, max_features=4, min_samples_leaf=2,
                      random_state=54321)

In [36]:
#calcular el valor rmse
predicted_test_dtr = model_dtr.predict(X_test_or_scaled)

model_dtr_rmse = rmse(predicted_test_dtr, y_test_or_scaled)

print('Valor de rmse en árboles de decisión:', model_dtr_rmse)

Valor de rmse en árboles de decisión: 0.4800306781652441


El modelo basado en árbol de decisión ocupa menos tiempo de ejecución que el bosque aleatorio, pero tiene un rmse un poco mayor, de 0.48.

### 2.1.3 Modelo Catboost

In [37]:
#crear un df con columnas numericas escaladas y columnas categoricas
data_cat_scaled = pd.concat([pd.DataFrame(data_num_scaled, columns=data_num_columns.columns)] + [data_categorical_columns], axis=1)

data_cat_scaled.head()

,Price,RegistrationYear,Power,Mileage,RegistrationMonth,NumberOfPictures,PostalCode,VehicleType,Gearbox,Model,FuelType,Brand,NotRepaired
0,-0.768404,-0.311174,-0.322499,0.612277,-0.051555,0.0,1.532972,small,manual,golf,petrol,volkswagen,no
1,-0.323343,0.824328,-0.365526,-1.005870,0.235819,0.0,0.347318,small,manual,fabia,gasoline,skoda,no
2,-0.948547,-1.284461,-0.128878,0.612277,1.097944,0.0,-0.684503,sedan,manual,3er,petrol,bmw,yes
3,-0.620050,0.175470,-0.078680,0.612277,0.523194,0.0,0.605641,convertible,manual,2_reihe,petrol,peugeot,no
4,-1.086304,-3.717679,-0.501778,-2.354325,0.235819,0.0,-1.242829,sedan,manual,other,petrol,volkswagen,no


In [38]:
#segmentar las caracteristicas y el objetivo
X_cat_scaled = data_cat_scaled.drop(['Price'], axis=1)
y_cat_scaled = data_cat_scaled['Price']

In [39]:
#segmentar en conjuntos de entrenamiento y prueba
X_train_cat_scaled, X_test_cat_scaled, y_train_cat_scaled, y_test_cat_scaled = func_train_test_split(X_cat_scaled, y_cat_scaled, 0.30)

In [40]:
#definir las caracteristicas categoricas en el set de entrenamiento
cat_features = [
    'VehicleType', 
    'Gearbox', 
    'Model', 
    'FuelType',
    'Brand', 
    'NotRepaired'
]

In [41]:
#construir un modelo catboost
model_cb = CatBoostRegressor(random_seed=42, loss_function='RMSE', iterations=100, learning_rate=0.8, verbose=10)

#entrenar el modelo
model_cb.fit(X_cat_scaled, y_cat_scaled, cat_features=cat_features)


0:	learn: 0.5822672	total: 162ms	remaining: 16s
10:	learn: 0.4051247	total: 903ms	remaining: 7.31s
20:	learn: 0.3863262	total: 1.68s	remaining: 6.31s
30:	learn: 0.3748849	total: 2.26s	remaining: 5.02s
40:	learn: 0.3665869	total: 2.84s	remaining: 4.09s
50:	learn: 0.3623705	total: 3.41s	remaining: 3.28s
60:	learn: 0.3584985	total: 3.97s	remaining: 2.54s
70:	learn: 0.3542786	total: 4.71s	remaining: 1.93s
80:	learn: 0.3516997	total: 5.39s	remaining: 1.26s
90:	learn: 0.3492917	total: 6.09s	remaining: 602ms
99:	learn: 0.3473875	total: 6.69s	remaining: 0us


In [42]:
#encontrando el valor rmse para el modelo de catboost
predicted_model_cb = model_cb.predict(X_test_cat_scaled)

model_cb_rmse = rmse(predicted_model_cb, y_test_cat_scaled)

print('Valor rmse para modelo Catboost:', model_cb_rmse)

Valor rmse para modelo Catboost: 0.34556041613367733


El modelo de catboost tarda un poco más que los modelos basados en árboles, pero el valor rmse es mucho más bajo.

### 2.1.4 Modelo LightGBM

## 3. Análisis

He construido 3 modelos de regresión para predecir los valores de mercado de los autos, ajustando los hiperparámetros con tres objetivos en mente que le interesan al cliente: la calidad de la predicción, la velocidad de la predicción, el tiempo requerido para el entrenamiento

* Regresión lineal

El modelo de regresión lineal resultó bastante rápido en cuanto al tiempo requerido para su entrenamiento y predicción, con valores aproximados de 7.7 y 0.3 segundos respectivamente. Sin embargo, en cuanto a la calidad de las predicciones, no es muy satisfactoria puesto que al calcular el RMSE para su evaluación, arrojó un valor de 0.57. Lo cual se puede mejorar con otros tipos de modelos.

* Bosque aleatorio

Este modelo requiere menos tiempo para el entrenamiento y para realizar las predcicciones, alrededor de 4.3 y 0.1 segundos respectivamente. La calidad de las predicciones también mejora bastante con respecto de la regresión lineal, con un valor RMSE de 0.44.

* Árbol de decisión

Este modelo mejoró en cuanto al tiempo requerido para entrenamiento y predicción (0.4s y 0.1s). Sin embargo, la calidad de sus predicciones es un poco más baja cuando se le compara con el modelo de bosque aleatorio, con un valor RMSE de 0.48.

* CatBoost

Este modelo fue configurado con una tasa de aprendizaje de 0.8 y 100 iteraciones. En vista de esto, el tiempo requerido para el entrenamiento aumenta mucho más con respecto a los modelos anteriores, a 7.5 segundos. En cuanto al tiempo que necesita para realizar las predicciones, es muy poco, alrededor de 0.1 segundos. En cuanto a la calidad de prediccón, supera al modelo de regresión y a los que están basados en árboles, con un RMSE de 0.34.
